In [27]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup

In [22]:
#url = 'http://services.wine.com/api/beta2/service.svc/json/reference?filter=filter=categories(490+124)&apikey=37032d5975318502c62227b58dcfe465'
url = 'http://www.wine.com/v6/Red-Wine/wine/list.aspx?N=7155+124&Nao=1'
req = requests.get(url)


In [29]:
soup = BeautifulSoup(req.text, 'lxml')

In [35]:
url_list = soup.find(attrs={'class':'productList'}).find_all('a', href=True)

In [36]:
len(url_list)

124

In [37]:
url_list[0]

<a href="/v6/Schug-Sonoma-Coast-Pinot-Noir-2014/wine/148901/Detail.aspx" id="ctl00_BodyContent_ctrProducts_rptResultsList_ctl00_ctrProduct_Label"><img alt="Schug Sonoma Coast Pinot Noir 2014" border="0" src="http://cache2.wine.com/labels/148901le.jpg"/></a>

In [26]:
tables = pd.read_html(req.text)

ValueError: No tables found

In [14]:
wines = json.loads(req.text)

In [18]:
wines.keys()

[u'Status', u'Blog', u'Vineyards', u'Books']

In [21]:
wines['Books']

[{u'Articles': [{u'Abstract': u"There is a countless number of red grape varieties in the world, some able to make wine, others best suited for Welch's grape juice. Right now, the world wine market focuses on about 40 to 50...",
    u'Content': u'',
    u'Footnotes': [],
    u'Id': u'124',
    u'Title': u'Red Wine',
    u'Url': u'http://www.wine.com/v6/Red-Wines/learnabout.aspx?class=1'},
   {u'Abstract': u'White wine differs from red wine in, first and most obviously, color. Under that skin, the pulpy part of a white grape is the same color as that of a red grape. The skin dictates the end color for...',
    u'Content': u'',
    u'Footnotes': [],
    u'Id': u'125',
    u'Title': u'White Wine',
    u'Url': u'http://www.wine.com/v6/White-Wines/learnabout.aspx?class=2'},
   {u'Abstract': u'\r\r\rRelated Links: \r\rShop our most popular Champagne and Sparkling Wine \r\rShop our highest rated Champagne and Sparkling Wine  \r\r\rThe essence of bubbles Many countries around the world make sp